In [64]:
import os
import sys
import pickle
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import torch
import pickle
from sklearn.preprocessing import LabelEncoder
from torch.optim import optimizer
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from torch.nn import CrossEntropyLoss,BCEWithLogitsLoss
from tqdm import tqdm_notebook, trange
import tqdm
from transformers import BertTokenizer, BertModel, BertConfig, BertForSequenceClassification
#from transformers.optimization import BertAdam, WarmupLinearSchedule
from sklearn.metrics import precision_recall_curve,classification_report
import matplotlib.pyplot as plt
%matplotlib inline

In [65]:
df = pd.read_csv('C:/Users/yhchou/Desktop/csv_file/nownews/news_content_0514.csv')
a= []
for title in df['title']:
    title = title.replace('| 政治 | NOWnews 今日新聞','')
    title = title.replace('| 影音 | NOWnews 今日新聞','')
    title = title.replace('| 節目 | NOWnews 今日新聞','')
    title = title.replace('| 娛樂 | NOWnews 今日新聞','')
    title = title.replace('| 焦點 | NOWnews 今日新聞','')
    title = title.replace('| 觀點 | NOWnews 今日新聞','')
    title = title.replace('| 國際 | NOWnews 今日新聞','')
    title = title.replace('| 運動 | NOWnews 今日新聞','')
    title = title.replace('| 房產 | NOWnews 今日新聞','')
    title = title.replace('| 財經 | NOWnews 今日新聞','')
    title = title.replace('| 兩岸 | NOWnews 今日新聞','')
    title = title.replace('| 新奇 | NOWnews 今日新聞','')
    title = title.replace('| 生活 | NOWnews 今日新聞','')
    title = title.replace('| 地方 | NOWnews 今日新聞','')
    a.append(title)

In [66]:
data = df.drop(['Unnamed: 0.1','content'], axis=1)
data['title'] = a

In [67]:
data

,Unnamed: 0,title,news_label
0,0,李來希扯「小燈泡」堅持不道歉！郭昱晴氣炸：你就是無恥,0
1,1,恐遭國民黨開除黨籍！李來希搬「46年經歷」狠嗆：我好怕,0
2,2,因選舉欠下2000萬全數還清 柯文哲：陳佩琪不會怪我了！,0
3,3,李來希挺韓失態扯小燈泡！她曝「最大受害者」：對立提升,0
4,4,白宮官員戴MIT口罩 外交部欣慰：抗疫期間發揮效益,0
...,...,...,...
471,233,盧秀燕市場當助手、致贈康乃馨慶祝母親節,13
472,234,新冠肺炎疫情趨緩 中市校園16日逐步開放,13
473,235,美食巷仔內／當季「馬頭魚」肥嫩鮮甜 新北網購買得到,13
474,236,避免棉絮飛舞 清總兵署進行木棉樹果摘除作業,13


In [68]:
data = data[~data['news_label'].isin([1])]
data = data[~data['news_label'].isin([2])]
data = data[~data['news_label'].isin([4])]
data = data[~data['news_label'].isin([5])]
data = data[~data['news_label'].isin([8])]


data['news_label'].replace(3,1,inplace=True)
data['news_label'].replace(6,2,inplace=True)
data['news_label'].replace(7,3,inplace=True)
data['news_label'].replace(9,4,inplace=True)
data['news_label'].replace(10,5,inplace=True)
data['news_label'].replace(11,6,inplace=True)
data['news_label'].replace(12,7,inplace=True)
data['news_label'].replace(13,8,inplace=True)

In [69]:
from sklearn.utils import shuffle
data= shuffle(data)

In [70]:
cut = 250
data =data.drop(['Unnamed: 0'], axis=1)

In [71]:
data_train = data.iloc[0:cut]

In [72]:
data_test = data.iloc[cut:306]

In [73]:
data_test

,title,news_label
369,NBA／字母哥四兄弟合體再等等 最小胞弟決定留在歐洲,3
125,中職／還我完全比賽！前兄弟洋投賈羅拉加向大聯盟討公道,3
435,房東不爽租客爆氣趕走！一開門全場嚇爛：我到底看了什麼,6
120,奧運／郭泰源對日本先發球威盡失 原來是這原因,3
126,中職／林安可砲火延燒國外 招牌辮子成外媒關注焦點,3
373,NBA／球隊黑幕是誰爆料的？喬丹在紀錄片指明是他,3
124,NBA／喬丹對待隊友近乎苛求 巴克利：他也是會挑人的,3
455,房東不爽租客爆氣趕走！一開門全場嚇爛：我到底看了什麼,7
474,避免棉絮飛舞 清總兵署進行木棉樹果摘除作業,8
441,韓國文化底蘊大輸日本？台灣人抖出「4關鍵原因」：難怪,6


In [74]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-chinese', do_lower_case=False)
# 封装类
class DataPrecessForSingleSentence(object):
    def __init__(self, bert_tokenizer, max_workers=10):
        """
        bert_tokenizer :分词器
        dataset        :包含列名为'text'与'label'的pandas dataframe
        """
        self.bert_tokenizer = bert_tokenizer
        # 创建多线程池
        self.pool = ThreadPoolExecutor(max_workers=max_workers)
        # 获取文本与标签
    def get_input(self, dataset, max_seq_len=30):
        """
        通过多线程（因为notebook中多进程使用存在一些问题）的方式对输入文本进行分词、ID化、截断、填充等流程得到最终的可用于模型输入的序列。
        
        入参:
            dataset     : pandas的dataframe格式，包含两列，第一列为文本，第二列为标签。标签取值为{0,1}，其中0表示负样本，1代表正样本。
            max_seq_len : 目标序列长度，该值需要预先对文本长度进行分别得到，可以设置为小于等于512（BERT的最长文本序列长度为512）的整数。
        
        出参:
            seq         : 在入参seq的头尾分别拼接了'CLS'与'SEP'符号，如果长度仍小于max_seq_len，则使用0在尾部进行了填充。
            seq_mask    : 只包含0、1且长度等于seq的序列，用于表征seq中的符号是否是有意义的，如果seq序列对应位上为填充符号，
                          那么取值为1，否则为0。
            seq_segment : shape等于seq，因为是单句，所以取值都为0。
            labels      : 标签取值为{0,1}，其中0表示负样本，1代表正样本。
        
            
        """
        sentences = dataset.iloc[:, 0].tolist()
        labels = dataset.iloc[:, 1].tolist()
        # 切词
        tokens_seq = list(
            self.pool.map(self.bert_tokenizer.tokenize, sentences))
        # 获取定长序列及其mask
        result = list(
            self.pool.map(self.trunate_and_pad, tokens_seq,
                          [max_seq_len] * len(tokens_seq)))
        seqs = [i[0] for i in result]
        seq_masks = [i[1] for i in result]
        seq_segments = [i[2] for i in result]
        return seqs, seq_masks, seq_segments, labels
    def trunate_and_pad(self, seq, max_seq_len):
        """
        1. 因为本类处理的是单句序列，按照BERT中的序列处理方式，需要在输入序列头尾分别拼接特殊字符'CLS'与'SEP'，
           因此不包含两个特殊字符的序列长度应该小于等于max_seq_len-2，如果序列长度大于该值需要那么进行截断。
        2. 对输入的序列 最终形成['CLS',seq,'SEP']的序列，该序列的长度如果小于max_seq_len，那么使用0进行填充。
        
        入参: 
            seq         : 输入序列，在本处其为单个句子。
            max_seq_len : 拼接'CLS'与'SEP'这两个特殊字符后的序列长度
        
        出参:
            seq         : 在入参seq的头尾分别拼接了'CLS'与'SEP'符号，如果长度仍小于max_seq_len，则使用0在尾部进行了填充。
            seq_mask    : 只包含0、1且长度等于seq的序列，用于表征seq中的符号是否是有意义的，如果seq序列对应位上为填充符号，
                          那么取值为1，否则为0。
            seq_segment : shape等于seq，因为是单句，所以取值都为0。
        """
        # 对超长序列进行截断
        if len(seq) > (max_seq_len - 2):
            seq = seq[0:(max_seq_len - 2)]
        # 分别在首尾拼接特殊符号
        seq = ['[CLS]'] + seq + ['[SEP]']
        # ID化
        seq = self.bert_tokenizer.convert_tokens_to_ids(seq)
        # 根据max_seq_len与seq的长度产生填充序列
        padding = [0] * (max_seq_len - len(seq))
        # 创建seq_mask
        seq_mask = [1] * len(seq) + padding
        # 创建seq_segment
        seq_segment = [0] * len(seq) + padding
        # 对seq拼接填充序列
        seq += padding
        assert len(seq) == max_seq_len
        assert len(seq_mask) == max_seq_len
        assert len(seq_segment) == max_seq_len
        return seq, seq_mask, seq_segment

# train data

In [75]:
processor = DataPrecessForSingleSentence(bert_tokenizer= bert_tokenizer)
# 产生输入ju 数据
seqs, seq_masks, seq_segments, labels = processor.get_input(dataset=data_train, max_seq_len=30)
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=9)
t_seqs = torch.tensor(seqs, dtype=torch.long)
t_seq_masks = torch.tensor(seq_masks, dtype = torch.long)
t_seq_segments = torch.tensor(seq_segments, dtype = torch.long)
t_labels = torch.tensor(labels, dtype = torch.long)

train_data = TensorDataset(t_seqs, t_seq_masks, t_seq_segments, t_labels)
train_sampler = RandomSampler(train_data)
train_dataloder = DataLoader(dataset= train_data, sampler= train_sampler,batch_size = 5)

# test data

In [76]:
processor = DataPrecessForSingleSentence(bert_tokenizer= bert_tokenizer)
# 产生输入ju 数据
te_seqs, te_seq_masks, te_seq_segments, te_labels = processor.get_input(dataset=data_test, max_seq_len=30)
#model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=14)
te_seqs = torch.tensor(te_seqs, dtype=torch.long)
te_seq_masks = torch.tensor(te_seq_masks, dtype = torch.long)
te_seq_segments = torch.tensor(te_seq_segments, dtype = torch.long)
te_labels = torch.tensor(te_labels, dtype = torch.long)

test_data = TensorDataset(te_seqs, te_seq_masks, te_seq_segments, te_labels)
test_sampler = RandomSampler(test_data)
test_dataloder = DataLoader(dataset= test_data, sampler= test_sampler,batch_size = 5)

In [77]:
model.train()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [78]:
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            
            # 別忘記前 3 個 tensors 分別為 tokens, segments 以及 masks
            # 且強烈建議在將這些 tensors 丟入 `model` 時指定對應的參數名稱
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions
    
# 讓模型跑在 GPU 上並取得訓練集的分類準確率
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)
_, acc = get_predictions(model, train_dataloder, compute_acc=True)
print("classification acc:", acc)


device: cuda:0
classification acc: 0.108


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-9)


EPOCHS = 150  # 幸運數字
for epoch in range(EPOCHS):
    
    running_loss = 0.0
    for data in train_dataloder:
        
        tokens_tensors, segments_tensors, \
        masks_tensors, labels = [t.to(device) for t in data]
        #print(labels)
        # 將參數梯度歸零
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)

        loss = outputs[0]
        # backward
        loss.backward()
        optimizer.step()


        # 紀錄當前 batch loss
        running_loss += loss.item()
        
    # 計算分類準確率
    _, acc_test = get_predictions(model, test_dataloder, compute_acc=True)
    _, acc_train = get_predictions(model, train_dataloder, compute_acc=True)

    print('[epoch %d] loss: %.3f, acc: %.3f' %
          (epoch + 1, running_loss, acc_test))
    print('[epoch %d] loss: %.3f, acc: %.3f' %
          (epoch + 1, running_loss, acc_train))

[epoch 1] loss: 10.186, acc: 0.625
[epoch 1] loss: 10.186, acc: 0.872
[epoch 2] loss: 10.076, acc: 0.625
[epoch 2] loss: 10.076, acc: 0.896
[epoch 3] loss: 10.162, acc: 0.625
[epoch 3] loss: 10.162, acc: 0.872
[epoch 4] loss: 10.198, acc: 0.643
[epoch 4] loss: 10.198, acc: 0.888
[epoch 5] loss: 10.050, acc: 0.661
[epoch 5] loss: 10.050, acc: 0.852
[epoch 6] loss: 10.046, acc: 0.643
[epoch 6] loss: 10.046, acc: 0.860
[epoch 7] loss: 10.532, acc: 0.643
[epoch 7] loss: 10.532, acc: 0.868
[epoch 8] loss: 10.027, acc: 0.643
[epoch 8] loss: 10.027, acc: 0.848
[epoch 9] loss: 10.277, acc: 0.625
[epoch 9] loss: 10.277, acc: 0.880
[epoch 10] loss: 10.500, acc: 0.607
[epoch 10] loss: 10.500, acc: 0.872
[epoch 11] loss: 10.292, acc: 0.643
[epoch 11] loss: 10.292, acc: 0.848
[epoch 12] loss: 10.289, acc: 0.625
[epoch 12] loss: 10.289, acc: 0.872
[epoch 13] loss: 10.053, acc: 0.607
[epoch 13] loss: 10.053, acc: 0.852
[epoch 14] loss: 10.107, acc: 0.661
[epoch 14] loss: 10.107, acc: 0.848
[epoch 15]

In [ ]:
#from transformers.optimization import AdamW

In [ ]:
#train_dataloder

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']

optimizer_grouped_parameters = [
    {
        'params':
        [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
        'weight_decay':
        0.01
    },
    {
        'params':
        [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
        'weight_decay':
        0.0
    }
]

optimizer = AdamW(optimizer_grouped_parameters,
                     lr=2e-05)

device = 'cpu'

# 存储每一个batch的loss
loss_collect = []
for i in trange(10, desc='Epoch'):
    for step, batch_data in enumerate(tqdm.notebook.tqdm_notebook(train_dataloder, desc='Iteration')):
        batch_data = tuple(t.to(device) for t in batch_data)
        batch_seqs, batch_seq_masks, batch_seq_segments, batch_labels = batch_data
        
        print(batch_seq_segments)
        # 对标签进行onehot编码
        one_hot = torch.zeros(batch_labels.size(0), 28).long()
        one_hot_batch_labels = one_hot.scatter_(
            dim=1,
            index=torch.unsqueeze(batch_labels, dim=1),
            src=torch.ones(batch_labels.size(0), 28).long())

        logits = model(batch_seqs, batch_seq_masks, batch_seq_segments, labels=None)
        logits = logits.softmax(dim=1)
        loss_function = CrossEntropyLoss()
        loss = loss_function(logits, batch_labels)
        loss.backward()
        loss_collect.append(loss.item())
        print("\r%f" % loss, end='')
        optimizer.step()
        optimizer.zero_grad()